In [37]:
class MatrixActions:
    def transponse(self, matrix: list):
        return tuple(tuple(matrix[j][i] for j in range(len(matrix))) for i in range(len(matrix[0])))


    def multiplication(self, matrix1: list, matrix2: list):
        return tuple(tuple(sum([matrix1[i][k] * matrix2[k][j]
                                for k in range(len(matrix2))])
                                for j in range(len(matrix2[0]))) 
                                for i in range(len(matrix1)))

    def matrix_subtraction(self, matrix1: tuple, matrix2: tuple):
        result = [[0 for j in range(len(matrix1[0]))] for i in range(len(matrix1))]
        for i in range(len(matrix1)):
            for j in range(len(matrix1[0])):
                result[i][j] = matrix1[i][j] - matrix2[i][j]
        result = tuple(tuple(j for j in i) for i in result)
        return result


    def multiply_matrix_on_number(self, number, matrix):
        return tuple(tuple(matrix[i][j] * number 
                           for j in range(len(matrix[i]))) 
                           for i in range(len(matrix)))

In [38]:
class ImageActions:
    def get_vector(self, matrix: tuple, height_of_block: int, width_of_block: int):
        y = 0
        result = []
        width = len(matrix[0])
        height = len(matrix)
        while y + width_of_block < height:
            x = 0
            while x + height_of_block < width:
                result.append(tuple(matrix[y + i][x + j] for i in range(width_of_block) for j in range(height_of_block)))
                x += height_of_block
            if x + height_of_block >= width:
                result.append(tuple(matrix[y + i][width - height_of_block + j] for i in range(width_of_block) for j in range(height_of_block)))
            y += width_of_block
        if y + width_of_block >= height:
            x = 0
            while x + height_of_block < width:
                result.append(tuple(matrix[height - width_of_block + i][x + j] for i in range(width_of_block) for j in range(height_of_block)))
                x += height_of_block
            if x + height_of_block >= width:
                result.append(tuple(matrix[height - width_of_block + i][width - height_of_block + j] for i in range(width_of_block) for j in range(height_of_block)))
        return tuple(result)


    def to_float(self, matrix: tuple):
        return tuple(tuple(color * 2 / 255 - 1 for j in i for color in j) for i in matrix)


    def to_color(self, matrix: tuple):
        return tuple(tuple(int((j + 1) / 2 * 255) for j in i) for i in matrix)


    def to_tuples(self, vector: tuple):
        return tuple(
            tuple(tuple((vector[i][j * 3 + color] for color in range(3))) 
                         for j in range(len(vector[i]) // 3)) 
                         for i in range(len(vector)))


    def to_pixels(self, matrix, height, width, n, m):
        buffer = []
        for i in range(height // m):
            for l in range(m):
                string = []
                for j in range(width // n):
                    for k in range(n):
                        string.append(matrix[i * width // m + j][k + l * m])
                buffer.append(string)
        result = []
        for i in buffer:
            result += i
        return tuple(result)


In [39]:
from random import uniform


class NeuralNetwork:
    def init_weights(self, height_of_block: int, width_of_block: int, neurons: int):
        weight1 = tuple(tuple(uniform(-1, 1) for _ in range(neurons)) for __ in range(height_of_block * width_of_block * 3))
        weight2 = tuple(tuple(uniform(-1, 1) for _ in range(height_of_block * width_of_block * 3)) for __ in range(neurons))
        return weight1, weight2

    def alpha(self, matrix: tuple):
        return 1 / sum([j ** 2 for i in matrix for j in i])

    def summary_error(self, matrix: tuple):
        return sum([j ** 2 for i in matrix for j in i])


In [40]:
from PIL import Image
from json import dump, load


MA = MatrixActions()
img_act = ImageActions()
NN = NeuralNetwork()

E = int(input("Max error: "))
image_name = input("Image name to load: ")
image = Image.open(f'mstis-1/{image_name}')
width, height = image.size
pixels = tuple(image.getdata())
matrix = tuple(tuple(pixels[i * width + j] for j in range(width)) for i in range(height))
match = int(input("0 - load weights from archieve\n1 - init new weights\nChoose:"))
if match == 0:
    n = int(input("Width of block: ")) 
    m = int(input("Height of block: "))
    neurons = int(input("Neurons number: "))
    with open(f"{n}x{m}x{neurons}.json", "r") as json_file:
        start_parametrs = load(json_file)
        W1 = tuple(tuple(i) for i in start_parametrs["weight1"])
        W2 = tuple(tuple(i) for i in start_parametrs["weight2"])
else:
    n = int(input("Width of block: "))
    m = int(input("Height of block: "))
    neurons = int(input("Neurons number: "))
    W1, W2 = NN.init_weights(n, m, neurons)
if n < width or m < height:
    squares = img_act.get_vector(matrix, n, m)
    X = img_act.to_float(squares)
    Y = MA.multiplication(X, W1)
    nX = MA.multiplication(Y, W2)
    delta_X = MA.matrix_subtraction(nX, X)
    W2 = MA.matrix_subtraction(W2, MA.multiplication(MA.multiply_matrix_on_number(NN.alpha(Y), MA.transponse(Y)), delta_X))
    W1 = MA.matrix_subtraction(W1, MA.multiplication(MA.multiplication(MA.multiply_matrix_on_number(NN.alpha(Y), MA.transponse(X)), delta_X), MA.transponse(W2)))
    counter = 1
    E1 = NN.summary_error(delta_X)
    while E < E1:
        E2 = E1
        print(counter, NN.summary_error(delta_X))
        Y = MA.multiplication(X, W1)
        nX = MA.multiplication(Y, W2)
        delta_X = MA.matrix_subtraction(nX, X)
        W2 = MA.matrix_subtraction(W2, MA.multiplication(MA.multiply_matrix_on_number(NN.alpha(Y), MA.transponse(Y)), delta_X))
        W1 = MA.matrix_subtraction(W1, MA.multiplication(MA.multiplication(MA.multiply_matrix_on_number(NN.alpha(Y), MA.transponse(X)), delta_X), MA.transponse(W2)))
        counter += 1
        E1 = NN.summary_error(delta_X)
        if abs(E2 - E1) < E2 * 0.001:
            break
else:
    print("It doesn't work")
pixels = img_act.to_color(nX)
pixels = img_act.to_tuples(pixels)
pixels = img_act.to_pixels(pixels, height, width, n, m)
out_image = Image.new(image.mode, image.size)
out_image.putdata(pixels)
out_image.show()
out_image.save(image_name)
with open(f"{n}x{m}x{neurons}.json", 'w+') as json_file:
    dump({"n": n, "m": m, "neurons": neurons, "weight1": W1, "weight2": W2}, json_file)
    

1 328642.4496108694
2 429395.47797499265
3 382141.33137603593
4 186280.64133410453
5 89170.6597493004
